In [1]:
from selenium.webdriver import Firefox
from selenium.webdriver.common.by import By
import sys, importlib

In [23]:
importlib.reload(sys.modules['pages.page_elements.fatura'])
# importlib.reload(sys.modules['scripts.preencher'])

from scripts.inicializar import Inicializar
from scripts.preencher_fatura import PreencherFatura
from pages.pages import PageFatura
# from pages.elements import *
from time import sleep


In [3]:
browser = Firefox()

pagina = Inicializar(
    browser,
    'https://guardiaov4.seplag.ce.gov.br/auth',
    'credenciais.txt'
).run()


KeyboardInterrupt



In [34]:
dados = {
    'Valor Adicional': 0.,
    'Valor Adicional Noturno': 15.,
    'Valor Reserva Técnica': 100.,
    'Valor Encargos': 1473.77,
    'Valor Insalubridade': 0.,
    'Valor Periculosidade': 0.,
    'Valor Outros': 0.,
    'Valor Vale Transporte': 150.,
    'Valor Vale Refeição': 0.,
}

In [24]:
p = PageFatura(browser)

In [25]:
p.terceirizados.carregar_funcionarios()
f = p.terceirizados.funcionarios

In [26]:
a = f[0]

In [28]:
a.ir_para_demais_informacoes()

In [ ]:
a.find_element(a.demais_informacoes._loc_tabela_montanteA).find_elements(By.TAG_NAME, 'tr')

In [35]:
a.demais_informacoes.preencher_montante_A(dados)
a.demais_informacoes.ir_para_montanteB()
a.demais_informacoes.preencher_montante_B(dados)

In [ ]:
tabela = a.find_element(a.demais_informacoes._loc_tabela_montanteA)

In [ ]:
for row in tabela.find_elements(By.TAG_NAME, 'tr'):
    descricao, valor = row.find_elements(By.TAG_NAME, 'td')
    descricao = descricao.text
    _loc_input = valor.find_element(By.TAG_NAME, 'input')
    valor = valor.find_element(By.TAG_NAME, 'input').get_attribute('value')
    print(descricao, valor, _loc_input)
    # print(row.find_element(By.TAG_NAME, 'input').get_attribute('value'))

In [ ]:
# pagina = PageFatura(browser)
p = Preencher(
    browser, 
    'data/SSPDS 024 (06-2022).xlsx', 
    intervalo_funcionarios=(13, 44), 
    nome_planilha='SPG' 
)

In [ ]:
dados = []

loc_box = '.table > tbody:nth-child(2) > tr:nth-child({}) > td:nth-child(7) > entrada-monetario:nth-child(1) > div:nth-child(1) > input:nth-child(1)'
to_click = '.table > tbody:nth-child(2) > tr:nth-child({}) > td:nth-child(6) > entrada-monetario:nth-child(1) > div:nth-child(1) > input:nth-child(1)'													
name = '.table > tbody:nth-child(2) > tr:nth-child({}) > td:nth-child(2) > div:nth-child(1) > span:nth-child(1)'

inicio = 21
for i in range(inicio, 101):
    # nome = browser.find_element(By.CSS_SELECTOR, '.table > tbody:nth-child(2) > tr:nth-child() > td:nth-child(2)').text
    print(f'{i}: {browser.find_element(By.CSS_SELECTOR, name.format(i)).text}: {dados[i-1]}')
    box = browser.find_element(By.CSS_SELECTOR, loc_box.format(i))
    box.clear()
    box.send_keys(dados[i-1])
    browser.find_element(By.CSS_SELECTOR, to_click.format(i)).click()
    sleep(7)